In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
from matplotlib import pyplot as plt
import numpy as np
import cv2
import glob

In [ ]:
model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
movenet = model.signatures['serving_default']

In [ ]:
path = '../input/choke-point/Choke Point Dataset/P2L_S2_C2.1'
keypoint_data1 = []
keypoint_data2 = []
keypoint_data3 = []
keypoint_data4 = []
keypoint_data5 = []
keypoint_data6 = []
for k in glob.glob(os.path.join(path,'*jpg')):
    img = cv2.imread(k)
    img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 224,224)
    input_img = tf.cast(img, dtype=tf.int32)
    results = movenet(input_img)
    keypoints_with_scores_1 = results['output_0'].numpy()[:,0,:51].reshape(51)
    keypoint_data1.append(keypoints_with_scores_1)
    keypoints_with_scores_1 = np.asarray( keypoint_data1)
    keypoints_with_scores_2 = results['output_0'].numpy()[:,1,:51].reshape(51)
    keypoint_data2.append(keypoints_with_scores_2)
    keypoints_with_scores_2 = np.asarray( keypoint_data2)
    keypoints_with_scores_3 = results['output_0'].numpy()[:,2,:51].reshape(51)
    keypoint_data3.append(keypoints_with_scores_3)
    keypoints_with_scores_3 = np.asarray( keypoint_data3)
    keypoints_with_scores_4 = results['output_0'].numpy()[:,3,:51].reshape(51)
    keypoint_data4.append(keypoints_with_scores_4)
    keypoints_with_scores_4 = np.asarray( keypoint_data4)
    keypoints_with_scores_5 = results['output_0'].numpy()[:,4,:51].reshape(51)
    keypoint_data5.append(keypoints_with_scores_5)
    keypoints_with_scores_5 = np.asarray(keypoint_data5)
    keypoints_with_scores_6 = results['output_0'].numpy()[:,5,:51].reshape(51)
    keypoint_data6.append(keypoints_with_scores_6)
    keypoints_with_scores_6 = np.asarray(keypoint_data6)

In [ ]:
import cv2
import glob

path = '../input/csa-train-set/OneDrive_1_05-04-2022'
data_set = []
frame_list = []
keypoints_with_scores_train = []
for k in glob.glob(os.path.join(path,'*mp4')):
    #frame_list = []
    cap = cv2.VideoCapture(k)
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break
    
    # Resize image
        img = frame.copy()
        img = img[680:2000, 400:680]
        #print(img.dtype)
        #input_img = tf.cast(img, dtype=tf.int32)
        #img = np.array(img)
        img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 224,224)
        input_img = tf.cast(img, dtype=tf.int32)
        
       
    # Detection section
        results = movenet(input_img)
        keypoints_with_scores_1 = results['output_0'].numpy()[:,0,:51].reshape(51)
        frame_list.append(keypoints_with_scores_1)
        keypoints_with_scores_1 = np.asarray(frame_list)
    #keypoints_with_scores_train = []
    keypoints_with_scores_train.append(keypoints_with_scores_1[30:120,:])
    keypoints_with_scores_train_np = np.asarray(keypoints_with_scores_train)
    print(keypoints_with_scores_1.shape)
    print(keypoints_with_scores_train_np.shape)
    
    #keypoints_with_scores_train.append(keypoints_with_scores_1[30:120,:])
    #keypoints_with_scores_train = []
    #for x in range(90,len(keypoints_with_scores_1)):
        #keypoints_with_scores_train.append(keypoints_with_scores_1[x-90:x,:])
    #keypoints_with_scores_train_np = np.asarray(keypoints_with_scores_train)
        #print(keypoints_with_scores_train_np.shape)
    #data_set.append(keypoints_with_scores_train)
    #data_set1 = np.asarray(data_set)
    #print(keypoints_with_scores_train_np.shape)

In [ ]:
import numpy as np
import keras as k
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(16, activation='relu', input_shape=(90,51), return_sequences=False))
model.add(RepeatVector(90))
model.add(LSTM(8, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(51)))
model.compile(optimizer='adam', loss=Autoencoder_loss,metrics=['mae',tf.keras.metrics.CosineSimilarity(axis=1)])
model.summary()

In [ ]:
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
callbacks =[tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=3),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.1,
                              patience=2, min_lr=0.0001)]

In [ ]:
train = keypoints_with_scores_train_np[0:74,:,:]
train.shape

In [ ]:
test = keypoints_with_scores_train_np[74:85,:,:]
test.shape

In [ ]:
history=model.fit(train,train,validation_data=(test,test),epochs=100,callbacks=callbacks,batch_size=8)

In [ ]:
model.save('/kaggle/working/autoencoder-model_justCSA5.h5')

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='best')
plt.show()

In [ ]:
plt.plot(history.history['cosine_similarity'])
plt.plot(history.history['val_cosine_similarity'])
plt.title('Model Cosine Similarity')
plt.ylabel('Cosine Similarity')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='best')
plt.show()

In [ ]:
keypoints_with_scores_train = []
for x in range(150,len(keypoints_with_scores_1)):
               keypoints_with_scores_train.append(keypoints_with_scores_1[x-90:x,:])

In [ ]:
keypoints_with_scores_train = np.asarray(keypoints_with_scores_train)
keypoints_with_scores_train.shape

In [ ]:
import keras.backend as K

In [ ]:
def Autoencoder_loss(x,x_hat):
    a,b,d = x.shape
    e,d,g = x_hat.shape
    x_hat = tf.reshape(x_hat,[e,90,17,3])
    x_hat = tf.cast(x_hat,tf.float32)
    print(x_hat.dtype)
    x = tf.reshape(x,[a,90,17,3])
    x = tf.cast(x,tf.float32)
    print(x.dtype)
    y_diff= K.square(x[:,:,:,1] - x_hat[:,:,:,1])
    x_diff = K.square(x[:,:,:,0] - x_hat[:,:,:,0])
    print(y_diff.dtype)
    print(x_diff.dtype)
    total_diff = y_diff+x_diff
    print("differnce",total_diff.shape)
    cof = x[:,:,:,2]
    product_list = []
    for l in range(len(cof[1,1,:])):
        total_diff = tf.transpose(total_diff)
        total_diff_che = total_diff[l,:,:]
        cof_che = cof[:,:,l]
        loss = K.dot(cof_che,total_diff_che)
        print("trans_differnce",loss.shape)
        loss = tf.linalg.diag_part(loss)
        loss = K.sum(loss)
        product_list.append(loss)
    print("coff",cof.shape)
    #loss = tf.tensordot(cof,total_diff,0)
    print(loss.shape)
    loss = no.array(product_list)
    loss = tf.linalg.diag_part(loss)
    print(loss.shape)
    loss = K.sum(loss)
    return loss

In [ ]:
model.save('/kaggle/working/autoencoder-model.pb')